In [3]:
import eventstudy as es
import numpy as np
import matplotlib.pyplot as plt

In [4]:
es.Single.import_returns('returns.csv')
es.Single.import_FamaFrench('famafrench.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'returns.csv'

In [ ]:
event = es.Single.FamaFrench_3factor(
    security_ticker = 'AAPL',
    event_date = np.datetime64('2013-03-04'),
    event_window = (-2,+10),
    estimation_size = 300,
    buffer_size = 30
)

In [ ]:
event.plot(AR=True)
plt.show()